In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

XPATH = By.XPATH

# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'

In [119]:
class Obligations_parser:
    def __init__(self, driver):
        self.driver = driver
        self.driver.maximize_window()
        self.driver.get(url)
        # self.obligations_data = pd.DataFrame(columns = ['Название', "Код", 'Ссылка', "Срок(месяца)", "Цена", "Номинал", "Выплат в год", "Величина купона(-13%)", "Прибыль в год(-13% с купона)", "Доходность(все время)", "Доходность(год)", "Амортизация", 'Рейтинг'])
        self.obligations_data = pd.DataFrame()
    

    def next_layer(self):
        next_page_class = 'Pagination-module__item_ALrvz Pagination-module__item_size_s_voSgR Pagination-module__item_arrow_mCpmW'
        try:
            self.driver.find_element(XPATH, f'//a[@class = "{next_page_class}"]').click()
            self.elements_iterator()
            return True

        except Exception:
            print('Все страницы пройдены.')
            return(self.obligations_data)
        


    
    def grab_paper_info(self, paper_url : str):
        self.driver.execute_script("window.open('');")
        self.driver.switch_to.window(self.driver.window_handles[-1])
        self.driver.get(paper_url)

        paper_data = {}

        paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
        
        paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})


        paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
        paper_time = paper_time[paper_time.index('з') + 3 : ]
        paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})
        del paper_time

        paper_data.update({'Цена' : [float(self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])]})
        
        paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

        paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')



        for counter in range(1, len(paper_datas) + 1, 2):
            match paper_datas[counter - 1].text:
                case "Величина купона":
                    paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                    paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                    paper_data.update({'Величина купона(-13%)' : [paper_coupon]})
                
                case "Номинал":
                    paper_data.update({'Номинал' : [float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
                
                case "Количество выплат в год":
                    paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                    paper_data.update({'Доходность(год)' : [paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})
                    # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                    paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                
                case 'Амортизация':
                    paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                

        print(paper_data)

        paper_data.update({'Ссылка' : [paper_url]})

        # print(pd.DataFrame(paper_data))
        # print(pd.DataFrame.from_dict(paper_data), )

        

        self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

        
        self.driver.close()

    def elements_iterator(self):

        link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            self.grab_paper_info(paper_link)

    
        return(self.obligations_data)

            

        # self.next_layer()



        # for paper in papers_rows:
        #     paper = paper.find_element(XPATH, '//td').find_element(XPATH, f'//a[@class = "{link_buttons_class}"]')
        #     print(paper.get_attribute('href'))




In [120]:
a = webdriver.Chrome()

In [121]:
parser_object = Obligations_parser(a)

In [122]:
df = parser_object.elements_iterator()

{'Название': ['Роснано выпуск 8'], 'Код': ['RU000A1008V9'], 'Срок(месяца)': [54], 'Цена': [864.0], 'Рейтинг': ['Средний'], 'Величина купона(-13%)': [51.33], 'Номинал': [1000.0], 'Выплат в год': [2], 'Доходность(год)': [102.66], 'Доходность(все время)': [1385.9099999999999]}
{'Название': ['Ferroni выпуск 1'], 'Код': ['RU000A103XP8'], 'Срок(месяца)': [24], 'Цена': [727.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [9.222], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [110.66399999999999], 'Доходность(все время)': [18.444]}
{'Название': ['Феррони БО-01'], 'Код': ['RU000A1053R3'], 'Срок(месяца)': [22], 'Цена': [907.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [11.745], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [140.94], 'Доходность(все время)': [11.745]}
{'Название': ['Агрофирма-племзавод Победа выпуск 1'], 'Код': ['RU000A102S23'], 'Срок(месяца)': [4], 'Цена': [937.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [21.141000000000002], 'Н

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class = "SecurityHeader__showName_iw6qC"]"}
  (Session info: chrome=117.0.5938.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF67E537892+54818]
	(No symbol) [0x00007FF67E4A6AC2]
	(No symbol) [0x00007FF67E35DA3B]
	(No symbol) [0x00007FF67E39E4FC]
	(No symbol) [0x00007FF67E39E67C]
	(No symbol) [0x00007FF67E3D9627]
	(No symbol) [0x00007FF67E3BEAEF]
	(No symbol) [0x00007FF67E3D75A2]
	(No symbol) [0x00007FF67E3BE883]
	(No symbol) [0x00007FF67E393691]
	(No symbol) [0x00007FF67E3948D4]
	GetHandleVerifier [0x00007FF67E89B992+3610402]
	GetHandleVerifier [0x00007FF67E8F1860+3962352]
	GetHandleVerifier [0x00007FF67E8E9D4F+3930847]
	GetHandleVerifier [0x00007FF67E5D3646+693206]
	(No symbol) [0x00007FF67E4B1628]
	(No symbol) [0x00007FF67E4AD934]


In [ ]:
df

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Роснано выпуск 8,RU000A1008V9,54,864.000000,Средний,51.330,1000.0,2,102.660,1385.910,https://www.tinkoff.ru/invest/bonds/RU000A1008V9/
0,Ferroni выпуск 1,RU000A103XP8,24,727.000000,Низкий,9.222,1000.0,12,110.664,18.444,https://www.tinkoff.ru/invest/bonds/RU000A103XP8/
0,Феррони БО-01,RU000A1053R3,22,907.000000,Низкий,11.745,1000.0,12,140.940,11.745,https://www.tinkoff.ru/invest/bonds/RU000A1053R3/
0,Агрофирма-племзавод Победа выпуск 1,RU000A102S23,4,937.000000,Низкий,21.141,1000.0,4,84.564,21.141,https://www.tinkoff.ru/invest/bonds/RU000A102S23/
0,Феррони БО-02,RU000A105P64,2,907.000000,Низкий,11.745,1000.0,12,140.940,0.000,https://www.tinkoff.ru/invest/bonds/RU000A105P64/
0,Alfa Bank,RU000A100GW5,11,835000.000000,Средний,254862.150,10000000.0,4,1019448.600,509724.300,https://www.tinkoff.ru/invest/bonds/RU000A100GW5/
0,М.Видео выпуск 4,RU000A106540,18,873.000000,23.04.2025,28.275,1000.0,4,113.100,113.100,https://www.tinkoff.ru/invest/bonds/RU000A106540/
0,МФК Эйрлоанс выпуск 1,RU000A1043L7,19,91.000000,Низкий,9.396,1000.0,12,112.752,9.396,https://www.tinkoff.ru/invest/bonds/RU000A1043L7/
0,М.Видео выпуск 3,RU000A104ZK2,21,87.000000,Низкий,26.709,1000.0,4,106.836,133.545,https://www.tinkoff.ru/invest/bonds/RU000A104ZK2/
0,Газпром Капитал БЗО26-1-Е,RU000A105QX1,28,70398.036201,Средний,3466.602,102245.3,1,3466.602,97064.856,https://www.tinkoff.ru/invest/bonds/RU000A105QX1/
